In [1]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [3]:
dfs = []
input_path = 'C:/Users/lzoeckler/Desktop/4plex/input_data/20190610'
for fname in os.listdir(input_path):
    plex_data = pd.read_csv('{}/{}'.format(input_path, fname), index_col=False,
                            skiprows=8, names=['patient_id', 'type', 'well', 'error',
                                               'HRP2_pg_ml', 'LDH_Pan_pg_ml',
                                               'LDH_Pv_pg_ml', 'CRP_ng_ml',
                                               'fail1', 'fail2'])
    plex_data.drop(['fail1', 'fail2'], axis=1, inplace=True)
    plex_data = plex_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    plex_data['patient_id'] = plex_data['patient_id'].fillna(method='ffill')
    plex_data = plex_data[~plex_data['patient_id'].isnull()]
    test = plex_data.loc[plex_data['patient_id'].str.contains('pa-014-63')]
    if len(test) > 1: 
        print(fname)
    dfs.append(plex_data)
combined = pd.concat(dfs)
combined = combined.loc[~combined['type'].isnull()]
combined = combined.loc[~combined['type'].str.contains('pixel')]
combined.head()

20190329_Data(Conc)_4Plex(Namibia P26).csv


,patient_id,type,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,CRP_ng_ml
0,calibrator (neat),reduced concentration (replicate 1),a1,NaN,600.41,11064.05,571.43,9423.69
1,calibrator (neat),reduced concentration (replicate 2),b1,NaN,483.68,9974.46,424.78,11129.36
2,calibrator (1:3),reduced concentration (replicate 1),a2,NaN,219.66,3501.87,181.60,3171.26
3,calibrator (1:3),reduced concentration (replicate 2),b2,NaN,196.66,3507.22,149.18,3156.21
4,calibrator (1:9),reduced concentration (replicate 1),a3,NaN,65.56,1208.19,57.82,990.08


In [6]:
check = pd.read_csv('{}/20190329_Data(Conc)_4Plex(Namibia P26).csv'.format(input_path), skiprows=8,
                    index_col=False)
check.head()

,calibrator (Neat),Reduced Concentration (Replicate 1),A1,MO3,Masked,10873.33,Masked.1,9301.11
0,NaN,Reduced Concentration (Replicate 2),B1,NaN,587.26,10157.26,497.04,11431.33
1,calibrator (1:3),Reduced Concentration (Replicate 1),A2,NaN,205.86,3573.70,165.30,3275.71
2,NaN,Reduced Concentration (Replicate 2),B2,NaN,188.22,3436.59,159.43,2943.16
3,calibrator (1:9),Reduced Concentration (Replicate 1),A3,NaN,67.53,1212.38,57.04,1025.78
4,NaN,Reduced Concentration (Replicate 2),B3,NaN,62.65,1116.72,Masked,1076.41


In [7]:
def fix_concentrations(df):
    con = df['concentration'].partition(':')[2]
    con = con.partition(')')[0]
    if len(con) != 0:
        return con
    else:
        return '1'

In [8]:
samples_data = combined.loc[combined['patient_id'].str.contains('pa-')]
samples_data = samples_data.drop('type', axis=1)
samples_data['concentration'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[-1])
samples_data['patient_id'] = samples_data['patient_id'].apply(lambda x: x.partition(' ')[0])
samples_data = samples_data.loc[(samples_data['concentration'].str.contains('neat|50'))]
samples_data = samples_data.loc[~samples_data['concentration'].str.contains('low volume')]
samples_data = samples_data.loc[~samples_data['well'].isnull()]
samples_data['concentration'] = samples_data.apply(fix_concentrations, axis=1)
samples_data = samples_data.sort_values(['patient_id', 'concentration'])
samples_data.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,CRP_ng_ml,concentration
33,pa-001-14,c8,NaN,299.48,< 14.41,2.76,7064.05,1
67,pa-001-14,d8,NaN,4138.79,< 720.50,< 116.50,3178.09,50
29,pa-001-21,c12,NaN,> 330.00,38.94,12.21,2413.65,1
63,pa-001-21,d12,NaN,1377.18,1593.58,227.59,< 1337.00,50
27,pa-001-28,g2,NaN,> 330.00,29.86,13.90,92.87,1


In [10]:
samples_data.loc[samples_data['patient_id'].str.contains('pa-014-63')]

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,CRP_ng_ml,concentration
26,pa-014-63,c10,NaN,< 1.07,19.01,7.59,> 9574.00,1
60,pa-014-63,d10,NaN,< 53.50,< 720.50,< 116.50,< 1337.00,50


In [11]:
samples_data['concentration'].unique()

array(['1', '50', '125000', '2500', '312500000', '6250000', '15625000000',
       '781250000000'], dtype=object)

In [85]:
sample_ids = samples_data['patient_id'].unique().tolist()
sample_set = set(sample_ids)

In [12]:
# threshhold values for various analytes
threshholds = {'HRP2_pg_ml': 330, 'LDH_Pan_pg_ml': 10514,
               'LDH_Pv_pg_ml': 497, 'CRP_ng_ml': 9574}

In [13]:
# constant to apply to the threshhold for different dilutions
dil_constants = {'50': 1, '2500': 50, '125000': 2500,
                 '6250000': 125000, '312500000': 6250000,
                 '15625000000': 312500000, '781250000000': 15625000000}

In [14]:
# positivity threshholds for various analytes
pos_threshholds = {'HRP2_pg_ml': 2.3, 'LDH_Pan_pg_ml': 47.8,
                   'LDH_Pv_pg_ml': 75.1, 'CRP_ng_ml': np.nan}

In [15]:
# dilution sets for various dilutions
dilution_sets = {'50': ('1', '50', 'fail'), '2500': ('50', '2500', 'fail'),
                 '125000': ('2500', '125000', 'fail'),
                 '6250000': ('125000', '6250000', 'fail'),
                 '312500000': ('6250000', '312500000', 'fail'),
                 '15625000000': ('312500000', '15625000000', 'fail'),
                 '781250000000': ('15625000000', '781250000000', 'fail')} 

In [16]:
duplicates = samples_data.loc[samples_data.duplicated(subset=['patient_id', 'concentration'], keep=False)]
deduped_dfs = []
for analyte in threshholds.keys():
    dup_analyte = duplicates[['patient_id', 'well', 'error', 'concentration', analyte]]
    pid_dfs = []
    for pid in duplicates['patient_id'].unique():
        dup_data = dup_analyte.loc[dup_analyte['patient_id'] == pid]
        con_dfs = []
        for concentration in dup_data['concentration'].unique():
            fill_df = pd.DataFrame(columns=['patient_id', 'well', 'error',
                                            'concentration', analyte])
            dup_con = dup_data.loc[dup_data['concentration'] == concentration]
            values = dup_con[analyte]
            wells = dup_con['well'].tolist()
            wells = ''.join(c for c in str(wells) if c not in ["[", "]", "'"])
            errors = dup_con['error'].tolist()
            non_nan_error = [e for e in errors if e is not np.nan]
            if not non_nan_error:
                errors = np.nan
            else:
                errors = non_nan_error
            try:
                values = [float(val) for val in values.tolist()]
                val = sum(values) / len(values)
            except ValueError:
                values = values.tolist()
                num_vals = [val for val in values if ('<' not in val) & ('>' not in val)]
                if len(num_vals) == 1:
                    val = num_vals[0]
                else:
                    val = values[0]
            fill_df = fill_df.append({'patient_id': pid, 'well': wells, 'error': errors,
                           'concentration': concentration, analyte: val}, ignore_index=True)
            con_dfs.append(fill_df)
        con_df = pd.concat(con_dfs)
        pid_dfs.append(con_df)
    pid_df = pd.concat(pid_dfs)
    deduped_dfs.append(pid_df)
deduped = reduce(lambda left, right: pd.merge(left, right, on=['patient_id', 'well', 'error', 'concentration']), deduped_dfs)
deduped.head()

,patient_id,well,error,concentration,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,CRP_ng_ml
0,pa-013-28,"c5, e1",NaN,1,117.59,56.865,6.515,2030.62
1,pa-013-28,"d5, f1",NaN,50,8823.61,1023.80,219.98,2194.03
2,pa-049-14,"d10, a8",NaN,125000,3.18711e+06,3.37477e+06,< 291250.00,< 3342500.00
3,pa-064-63,"a9, e4",NaN,1,205.16,22.72,11.85,27.64
4,pa-064-63,"b9, f4",NaN,50,1424.38,< 720.50,< 116.50,< 1337.00


In [17]:
no_duplicates = samples_data.drop_duplicates(subset=['patient_id', 'concentration'], keep=False)
no_duplicates = pd.concat([no_duplicates, deduped])
no_duplicates.head()

C:\Users\lzoeckler\miniconda3\envs\DIAMETER\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,CRP_ng_ml,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,concentration,error,patient_id,well
33,7064.05,299.48,< 14.41,2.76,1,NaN,pa-001-14,c8
67,3178.09,4138.79,< 720.50,< 116.50,50,NaN,pa-001-14,d8
29,2413.65,> 330.00,38.94,12.21,1,NaN,pa-001-21,c12
63,< 1337.00,1377.18,1593.58,227.59,50,NaN,pa-001-21,d12
27,92.87,> 330.00,29.86,13.90,1,NaN,pa-001-28,g2


In [18]:
no_duplicates.loc[no_duplicates['patient_id'].str.contains('pa-014-63')]

,CRP_ng_ml,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,concentration,error,patient_id,well
26,> 9574.00,< 1.07,19.01,7.59,1,NaN,pa-014-63,c10
60,< 1337.00,< 53.50,< 720.50,< 116.50,50,NaN,pa-014-63,d10


In [19]:
for err in no_duplicates['error'].unique():
    print(err)
    print(type(err))

nan
<class 'float'>
software unable to process full dilution, the sample was run as neat and the data for this sample was multiplied by the dilution factor (1:15625000000) in this data file only. 
<class 'str'>
software unable to process full dilution, the sample was run as neat and the data for this sample was multiplied by the dilution factor (1:781250000000) in this data file only. 
<class 'str'>


In [20]:
def return_decisions(low, high, fail='fail'):
    # Columns = [neat_above, neat_below, neat_LLQ, neat_ULQ, NA]
    # Rows = [dil_above, dil_below, dil_LLQ, dil_ULQ, NA]
    HRP2_matrix = np.array([[high, high, high, high, high],
                            [high, low, low, high, fail],
                            [high, low, low, fail, fail],
                            [high, high, fail, high, high],
                            [fail, high, high, fail, fail]])

    other_matrix = np.array([[high, low, low, high, high],
                               [high, low, low, high, fail],
                               [high, low, low, fail, fail],
                               [high, low, fail, high, high],
                               [fail, low, low, fail, fail]])

    # decisions for various analytes
    decisions = {'HRP2_pg_ml': HRP2_matrix, 'LDH_Pan_pg_ml': other_matrix,
                 'LDH_Pv_pg_ml': other_matrix, 'CRP_ng_ml': other_matrix}
    return(decisions)

In [21]:
def run_compare(df, analyte_val, dil_val):
    above, below, LLQ, ULQ, NA = False, False, False, False, False
    val = df[analyte_val]
    thresh_val = dil_constants[dil_val] * threshholds[analyte_val]
    try:
        float_val = float(val)
        if math.isnan(float_val):
            NA = True
        elif float_val > thresh_val:
            above = True
        elif float_val < thresh_val:
            below = True
    except ValueError:
        if '<' in val:
            LLQ = True
        elif '>' in val:
            ULQ = True
    finally:
        return(np.array([above, below, LLQ, ULQ, NA]))

In [87]:
analyte_dfs = []
for analyte in threshholds.keys():
# for analyte in ['CRP_ng_ml']:
    patient_dfs = []
    for i in no_duplicates['patient_id'].unique():
#     for i in ['pa-014-63']:
        patient_data = no_duplicates.loc[no_duplicates['patient_id'] == i]
        dilution_values = sorted([val for val in patient_data['concentration'].unique() if val != '1'], key=len)
        best_decision = '1'
        for max_dilution in dilution_values:
#         for max_dilution in ['15625000000']:
            dil_data = patient_data.loc[patient_data['concentration'].isin([best_decision, max_dilution])]          
            partial_compare = partial(run_compare, analyte_val=analyte, dil_val=max_dilution)
            dil_data['decision_vector'] = dil_data.apply(partial_compare, axis=1)
            decisions = return_decisions(best_decision, max_dilution)
            decision_matrix = decisions[analyte]
            best_df = pd.DataFrame(columns=['patient_id', 'errors', analyte,
                                            '{}_dilution'.format(analyte),
                                            '{}_well'.format(analyte),
                                            '{}_max_dilution'.format(analyte)])
            vector_low = dil_data.loc[dil_data['concentration'] == best_decision,
                                      'decision_vector'].item()
            vector_high = dil_data.loc[dil_data['concentration'] == max_dilution,
                                       'decision_vector'].item()
            decision = decision_matrix[vector_high, vector_low].item()
            if decision in [best_decision, max_dilution]:
                val = dil_data.loc[dil_data['concentration'] == decision,
                                   analyte].item()
                well = dil_data.loc[dil_data['concentration'] == decision,
                                    'well'].item()
                error = dil_data.loc[dil_data['concentration'] == decision,
                                    'error'].item()
            elif decision == 'fail':
                val = 'fail'
                well = 'fail'
                error = np.nan
            else:
                raise ValueError("Unexpected decision value: {}".format(decision))
            other_dilutions = [val for val in patient_data['concentration'].unique()]
            other_dilutions = [float(val) for val in other_dilutions if val != 'fail']
            max_dilution = int(max(other_dilutions))
            df_decision = decision if decision != 'fail' else np.nan
            best_decision = decision
            best_df = best_df.append({'patient_id': i, 'errors': error, analyte: val,
                                      '{}_dilution'.format(analyte): df_decision,
                                      '{}_well'.format(analyte): well,
                                      '{}_max_dilution'.format(analyte): max_dilution}, ignore_index=True)
            if decision == 'fail':
                break
        patient_dfs.append(best_df)
    patient_df = pd.concat(patient_dfs)
    patient_df['errors'] = patient_df['errors'].astype('object')
    analyte_dfs.append(patient_df)
final_df = reduce(lambda left, right: pd.merge(left, right, on=['patient_id', 'errors']), analyte_dfs)

C:\Users\lzoeckler\miniconda3\envs\DIAMETER\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [88]:
final_df.head()

,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_max_dilution,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,LDH_Pan_pg_ml_well,LDH_Pan_pg_ml_max_dilution,LDH_Pv_pg_ml,LDH_Pv_pg_ml_dilution,LDH_Pv_pg_ml_well,LDH_Pv_pg_ml_max_dilution,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_max_dilution
0,pa-001-14,NaN,4138.79,50,d8,50,< 14.41,1,c8,50,2.76,1,c8,50,7064.05,1,c8,50
1,pa-001-21,NaN,1377.18,50,d12,50,38.94,1,c12,50,12.21,1,c12,50,2413.65,1,c12,50
2,pa-001-28,NaN,2024.41,50,h2,50,29.86,1,g2,50,13.90,1,g2,50,92.87,1,g2,50
3,pa-001-3,NaN,318702.59,2500,e3,312500000,142.30,1,g6,312500000,15.35,1,g6,312500000,5678.58,1,g6,312500000
4,pa-001-7,NaN,73737.79,2500,c5,125000,179.34,1,e12,125000,10.31,1,e12,125000,1021.09,1,e12,125000


In [78]:
final_ids = final_df['patient_id'].unique().tolist()
final_set = set(final_ids)
assert final_set == sample_set, print(sample_set - final_set)

In [ ]:
final_ids

In [79]:
def split_time(df):
    sub = df['patient_id'].split('-')
    try:
        time = int(sub[2])
        return time
    except IndexError:
        return 0

In [80]:
def remove_time(df):
    patient = df['patient_id'].split('-')
    return '{}-{}'.format(patient[0], patient[1])

In [81]:
time_df = final_df.copy(deep=True)
time_df['time_point_days'] = time_df.apply(split_time, axis=1)
time_df['patient_id'] = time_df.apply(remove_time, axis=1)
time_df.sort_values(['patient_id', 'time_point_days'], inplace=True)
time_df.set_index(['patient_id', 'time_point_days'], inplace=True)
time_df.head()

errors HRP2_pg_ml HRP2_pg_ml_dilution  \
patient_id time_point_days                                         
pa-001     3                  NaN  318702.59                2500   
           7                  NaN   73737.79                2500   
           14                 NaN    4138.79                  50   
           21                 NaN    1377.18                  50   
           28                 NaN    2024.41                  50   

                           HRP2_pg_ml_well  HRP2_pg_ml_max_dilution  \
patient_id time_point_days                                            
pa-001     3                            e3              312500000.0   
           7                            c5                 125000.0   
           14                           d8                     50.0   
           21                          d12                     50.0   
           28                           h2                     50.0   

                           LDH_Pan_pg_ml LDH_Pan_pg_ml_dilution  \
patient_id time_point_days                                        
pa-001     3                      142.30                      1   
           7                      179.34                      1   
           14                    < 14.41                      1   
           21                      38.94                      1   
           28                      29.86                      1   

                           LDH_Pan_pg_ml_well  LDH_Pan_pg_ml_max_dilution  \
patient_id time_point_days                                                  
pa-001     3                               g6                 312500000.0   
           7                              e12                    125000.0   
           14                              c8                        50.0   
           21                             c12                        50.0   
           28                              g2                        50.0   

                           LDH_Pv_pg_ml LDH_Pv_pg_ml_dilution  \
patient_id time_point_days                                      
pa-001     3                      15.35                     1   
           7                      10.31                     1   
           14                      2.76                     1   
           21                     12.21                     1   
           28                     13.90                     1   

                           LDH_Pv_pg_ml_well  LDH_Pv_pg_ml_max_dilution  \
patient_id time_point_days                                                
pa-001     3                              g6                312500000.0   
           7                             e12                   125000.0   
           14                             c8                       50.0   
           21                            c12                       50.0   
           28                             g2                       50.0   

                           CRP_ng_ml CRP_ng_ml_dilution CRP_ng_ml_well  \
patient_id time_point_days                                               
pa-001     3                 5678.58                  1             g6   
           7                 1021.09                  1            e12   
           14                7064.05                  1             c8   
           21                2413.65                  1            c12   
           28                  92.87                  1             g2   

                            CRP_ng_ml_max_dilution  
patient_id time_point_days                          
pa-001     3                           312500000.0  
           7                              125000.0  
           14                                 50.0  
           21                                 50.0  
           28                                 50.0

In [82]:
time_df.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/final_dilutions.csv')

In [83]:
partial_format = samples_data.copy(deep=True)
partial_format['time_point_days'] = partial_format.apply(split_time, axis=1)
partial_format['patient_id'] = partial_format.apply(remove_time, axis=1)
partial_format.sort_values(['patient_id', 'time_point_days'], inplace=True)
partial_format.set_index(['patient_id', 'time_point_days'], inplace=True)
partial_format.head()

well error     HRP2_pg_ml    LDH_Pan_pg_ml  \
patient_id time_point_days                                              
pa-001     3                 g6   NaN         145.95           142.30   
           3                 f3   NaN      397468.45     < 1801250.00   
           3                 e3   NaN      318702.59         74360.14   
           3                 h3   NaN  1537195777.01  119019730410.15   
           3                 h6   NaN       10273.61          2535.11   

                              LDH_Pv_pg_ml        CRP_ng_ml concentration  
patient_id time_point_days                                                 
pa-001     3                         15.35          5678.58             1  
           3                   < 291250.00     < 3342500.00        125000  
           3                      10385.14       < 66850.00          2500  
           3                16185275422.09  286153421264.94     312500000  
           3                        733.44         49491.76            50

In [84]:
partial_format.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/partially_formatted.csv')